# 할리스 매장정보 웹스크래핑 실습

In [ ]:
# 1. 필요한 라이브러리 설치 및 임포트
!pip install requests beautifulsoup4 pandas

## 0. 웹 페이지 분석하기
- 웹 크롤링을 시작하기 전에 대상 웹사이트의 구조를 분석해야 합니다.

### 0.1 URL 엔드포인트 분석
- **대상 URL**: `https://www.hollys.co.kr/store/korea/korStore2.do`
- **HTTP 메서드**: GET
- **설명**: 할리스 커피 매장 정보를 제공하는 페이지

### 0.2 요청 파라미터 분석
- **pageNo**: 페이지 번호 (1부터 시작)
  - 예시: `?pageNo=1`, `?pageNo=2`
  - 각 페이지마다 여러 매장 정보를 포함

### 0.3 HTML 구조 분석 (개발자 도구 활용)

#### 브라우저 개발자 도구 사용법:
1. 웹 페이지 접속 후 개발자 도구 열기
    - **Windows**: `F12` 또는 `Ctrl + Shift + I` 또는 `우클릭 > 검사`
    - **Mac**: `Cmd + Option + I` 또는 `우클릭 > 검사`
2. Elements 탭에서 원하는 데이터 위치를 찾아 HTML 구조 확인

#### 매장 정보 테이블 구조:
```html
<table class="tb_store">
    <thead>
        <tr>
            <th>지역</th>
            <th>매장명</th>
            <th>영업상태</th>
            <th>주소</th>
            <th>서비스</th>
            <th>전화번호</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>서울</td>
            <td>강남점</td>
            <td>영업중</td>
            <td>서울시 강남구 ...</td>
            <td><img alt="와이파이" /></td>
            <td>02-1234-5678</td>
        </tr>
        <!-- 더 많은 매장 정보 ... -->
    </tbody>
</table>
```

### 0.4 CSS 선택자 경로
- **테이블 전체**: `table.tb_store`
- **모든 매장 행**: `table.tb_store > tbody > tr`
- **개별 셀 구조**:
  - `td:nth-child(1)`: 지역
  - `td:nth-child(2)`: 매장명
  - `td:nth-child(3)`: 영업상태
  - `td:nth-child(4)`: 주소
  - `td:nth-child(5)`: 서비스 (이미지)
  - `td:nth-child(6)`: 전화번호

### 0.5 서비스 아이콘 분석
- 서비스 정보는 `<img>` 태그의 `alt` 속성에 저장
- 예시: `<img alt="와이파이" />`, `<img alt="주차" />`
- 여러 서비스가 있을 경우 공백으로 결합

### 0.6 요청 헤더 설정
- **User-Agent** 설정 필요
  - 일부 웹사이트는 User-Agent가 없는 요청을 차단
  - 실제 브라우저처럼 보이도록 설정

## 1. 웹 페이지 요청하기
- 웹사이트에서 데이터를 가져오기

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from typing import List, Dict
import os
import sys
from pprint import pprint

def get_hollys_store_info(page: int) -> str:
    """
    할리스 매장 정보 페이지의 HTML을 가져오는 함수
    
    Args:
        page (int): 페이지 번호
    Returns:
        str: 웹 페이지의 HTML 내용
    """
    url = "https://www.hollys.co.kr/store/korea/korStore2.do"
    params = {"pageNo": page}
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/120.0.0.0'
    }
    
    try:
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()  # 오류가 있으면 예외를 발생시킴
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"페이지 {page} 요청 중 에러 발생: {e}")
        return ""

In [ ]:
# 테스트: 첫 페이지 가져오기
html = get_hollys_store_info(1)
print("HTML 일부:", html[:500])  # 처음 500자만 출력

## 2. HTML 파싱하기
- BeautifulSoup를 사용하여 HTML에서 필요한 정보를 추출

In [ ]:
def parse_store_info(html: str) -> List[Dict]:
    """
    HTML에서 매장 정보를 파싱하는 함수
    
    Args:
        html (str): 파싱할 HTML 문자열
    Returns:
        List[Dict]: 매장 정보 딕셔너리의 리스트
    """
    stores = []
    soup = BeautifulSoup(html, 'html.parser')
    
    # CSS 선택자로 table.tb_store > tbody > tr 선택
    rows = soup.select("table.tb_store > tbody > tr")
    
    # find() 함수 이용 단일 태그로 찾기
    # table = soup.find("table", class_="tb_store")
    # tbody = table.find("tbody")
    # rows = tbody.find_all("tr")
   
    for tr in rows:
        tds = tr.find_all('td')
        if len(tds) < 6:
            continue
        
        store = {
            'region': tds[0].text.strip(),
            'name': tds[1].text.strip(),
            'status': tds[2].text.strip(),
            'address': tds[3].text.strip(),
            'service': ' '.join(img['alt'] for img in tds[4].find_all('img')),
            'phone': tds[5].text.strip()
        }
        stores.append(store)
    
    return stores

In [ ]:
# 테스트: 첫 페이지 파싱하기
stores = parse_store_info(html)
print(f"첫 페이지에서 찾은 매장 수: {len(stores)}")
print("\n첫 번째 매장 정보:")
pprint(stores[0])

## 3. 데이터 저장하기
- 수집한 데이터를 CSV와 JSON 형식으로 저장

In [ ]:
def save_to_files(stores: List[Dict], base_path: str = "./웹크롤링/커피매장"):
   """
   매장 정보를 CSV와 JSON 파일로 저장하는 함수
   """
   
   # 디렉토리 생성 (존재하지 않으면 생성)
   os.makedirs(base_path, exist_ok=True)
   
   # DataFrame 생성
   df = pd.DataFrame(stores)
   
   # CSV 파일로 저장
   file_name = "hollys_stores.csv"
   csv_file_path = os.path.join(base_path, file_name)
   df.to_csv(csv_file_path, encoding='utf-8', index=False)
   
   # JSON 파일로 저장 (DataFrame 활용)
   file_name = f"hollys_stores.json"
   json_file_path = os.path.join(base_path, file_name)
   df.to_json(json_file_path, orient='records', force_ascii=False, indent=4)
   
   print(f"CSV 파일 저장 완료: {csv_file_path}")
   print(f"JSON 파일 저장 완료: {json_file_path}")
   
   return df

In [ ]:
# 테스트: 첫 페이지 데이터 저장하기
df = save_to_files(stores)
display(df)

## 5. 전체 과정 실행하기

In [ ]:
stores = []

# 매장 정보 수집
print("할리스 매장 정보 수집 중...")
for page in range(1, 4):  # 1~3 페이지만 수집 (테스트용)
    html = get_hollys_store_info(page)
    if html:
        page_stores = parse_store_info(html)
        stores.extend(page_stores)
        print(f"페이지 {page}: {len(page_stores)}개의 매장 정보 수집 완료")

if not stores:
    print("매장 정보를 가져오는데 실패했습니다.")
    sys.exit(1)
    
print(f"\n총 {len(stores)}개의 매장 정보를 수집했습니다.")

# 데이터 저장
try:
    save_to_files(stores)
except Exception as e:
    print(f"데이터 저장 중 에러 발생: {e}")